# Evaluate the fidelity

In [3]:
from Datasets.synthetics import Infection
import torch
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Setup PyG

In [4]:
torch.set_num_threads(4)
device = 'cuda'

from metrics.utils_nc import set_seeds

# Compute the fidelity

In [5]:
from metrics.utils_nc import build_expl, compute_fidelity
from models.models_Infection import GCN_framework, GraphSAGE_framework, GAT_framework, GIN_framework, CHEB_framework


# Define the parameters
DATASET = 'Infection'
MODELS = ['GCN', 'GAT', 'GIN', 'GraphSAGE', 'Cheb']

EXPLS = ['cam', 'grad_cam', 'grad_exp', 'guided_bp', 'ig_node', 'pgmexplainer', 'gnnexpl', 'pgexplainer']
EXPLS = ["subgraphX"]
MODES = ['train']

IGNORE = [('Infection', 'Cheb', 'gnnexpl', 'train'),
          ('Infection', 'Cheb', 'pgexplainer', 'train'),
          ('Infection', 'GIN', 'gnnexpl', 'train')]

GNN_NUM_LAYERS = {'GCN': 2, 'GAT': 2, 'GIN': 2, 'GraphSAGE': 2, 'Cheb': 2}

FRAMEWORKS = {'GCN': GCN_framework, 
              'GAT': GAT_framework, 
              'GIN': GIN_framework,
              'GraphSAGE': GraphSAGE_framework, 
              'Cheb': CHEB_framework
             }


# Load the dataset
set_seeds()
dataset = Infection()


# Define history variables
suff = {0: [], 1: [], 2: []}
model, expl = [], []


# Compute the metrics
for MODE in MODES:
    for MODEL in MODELS:
        print(8 * '* ' + MODEL + 8 * ' *')
        
        # Define and load the trained model
        gcn = FRAMEWORKS[MODEL](dataset, device='cuda')
        path = 'models/' + DATASET + '_' + MODEL
        gcn.load_model(path)

        # Loop over the explainers
        for EXPL in EXPLS:
            # Define the setting and store it
            ID = (DATASET, MODEL, EXPL, MODE)
            model += [MODEL]
            expl += [EXPL]
            
            # Compute the sufficiency
            if ID in IGNORE:
                # Return nan metrics
                suff[0].append(float('nan'))
                suff[1].append(float('nan'))
                suff[2].append(float('nan'))
            else:
                # Load and process the explanations
                graphs = build_expl(DATASET, MODEL, EXPL, GNN_NUM_LAYERS, num_features=2, cut_ego=False)

                # Loop over the class labels
                for label in graphs:
                    if not graphs[label] == None:
                        # If there are valid explanations compute suff and comp    
                        suff[label] += [compute_fidelity(gcn, dataset.data, graphs, num_features=2, y=label)]
                    else:
                        # Otherwise return nan metrics
                        suff[label].append(float('nan'))
                    
            # Print the partial results                    
            print(' '.join([' {:.3f}'.format(suff[label][-1]) if ~np.isnan(suff[label][-1]) else ' -----' for label in suff]) + '\t' + EXPL)        

* * * * * * * * GCN * * * * * * * *


/home/antonio/anaconda3/envs/geo_new/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/antonio/anaconda3/envs/geo_new/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


 0.997  0.760  -0.198	subgraphX
* * * * * * * * GAT * * * * * * * *
 -0.297  0.771  0.657	subgraphX
* * * * * * * * GIN * * * * * * * *
 0.988  0.771  -0.200	subgraphX
* * * * * * * * GraphSAGE * * * * * * * *
 0.642  -0.191  0.774	subgraphX
* * * * * * * * Cheb * * * * * * * *
 -----  -----  -----	subgraphX


In [7]:
import pandas as pd
results = pd.DataFrame({'model': model, 'expl': expl, 'class 0': suff[0], 'class 1': suff[1], 'class 2': suff[2]})
results.fillna(-100).to_csv('./metrics/fidelity_sufficiency_' + '_'.join(MODELS) + '.csv')
results

,model,expl,class 0,class 1,class 2
0,GCN,subgraphX,0.997460,0.759996,-0.197984
1,GAT,subgraphX,-0.296510,0.771209,0.657493
2,GIN,subgraphX,0.988424,0.770655,-0.199946
3,GraphSAGE,subgraphX,0.641560,-0.190920,0.774214
4,Cheb,subgraphX,NaN,NaN,NaN
